In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import scale
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns
import math
import time
import datetime
from pandas.tseries.offsets import *
from xgboost import XGBRegressor
from sklearn import preprocessing
from sklearn import linear_model

from pylab import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
pd.options.display.max_rows = 999

In [124]:
%%time
flat = pd.read_csv('flat_fix.csv', parse_dates=['sale','date_salestart', 'date_settle','flat_startsale'])
price = pd.read_csv('price.csv',encoding='cp1251',parse_dates=['datefrom','dateto','date_salestart'])
status = pd.read_csv('status.csv',encoding='cp1251',parse_dates=['datefrom','dateto'])
test = pd.read_csv('test.csv', parse_dates=['date1'],encoding='cp1251')
train = pd.read_csv('train.csv', parse_dates=['date1'],encoding='cp1251')

y = train.value

train = train.drop(['value','id','start_square','plan_s','plan_m','plan_l','vid_0','vid_1','vid_2'], axis=1)
test = test.drop(['id'], axis=1)

columns = ['id_bulk', 'spalen', 'date1', 'price', 'mean_sq', 'mean_fl','month', 'month_cnt', 'objclass', 'roomcount', 'wall', 'area', 'vh_groups', 'kindergarten', 'school',
           'hospital', 'phok', 'trainarea', 'carwash', 'stores', 'wheelchairs', 'conditioner', 'ventilation', 'elevator', 'garbage_col', 'cctv', 'u_parking', 
           'noauto_yard', 'auto_places', 'factory_zone', 'green_zone', 'to_kremlin', 'to_mall', 'to_sadoviy', 'to_bigroad', 'to_autocross', 'to_undeground_walk',
           'to_factory', 'to_park', 'to_park_walk', 'u_station_ring', 'yard_area', 'dollar', 'mortgage', 'deposit1', 'deposit13', 'depostit3']

train.columns = columns
test.columns = columns

train['value'] = y

train = pd.concat([train, pd.get_dummies(train['objclass'],prefix='c')], axis=1)
test = pd.concat([test, pd.get_dummies(test['objclass'],prefix='c')], axis=1)

for i in ['wall', 'vh_groups','carwash', 'stores', 'wheelchairs', 'u_parking', 'noauto_yard']:
    train[i] = train[i].replace('да', 1).replace('нет', 0)
    test[i] = test[i].replace('да', 1).replace('нет', 0)
    
train = train.drop(['garbage_col','elevator','trainarea', 'objclass'], axis=1)
test = test.drop(['garbage_col','elevator','trainarea', 'objclass'], axis=1)

train = pd.concat([train, pd.get_dummies(train['month'],prefix='m')], axis=1).drop(['month'], axis=1)
months = pd.concat([pd.get_dummies(test['month'],prefix='m'), pd.DataFrame(data = np.zeros((test.shape[0],9)),\
                columns = ['m_1','m_2','m_6','m_7','m_8','m_9','m_10','m_11','m_12'])], axis=1).sort_index(axis=1)
test = pd.concat([test, months], axis=1).drop(['month'], axis=1)

train['id_bulk_sp'] = train['id_bulk'].map(str)+'_'+train['spalen'].map(str)
test['id_bulk_sp'] = test['id_bulk'].map(str)+'_'+test['spalen'].map(str)
flat['id_bulk_sp'] = flat['bulk_id'].map(str)+'_'+flat['spalen'].astype(int).map(str)

flat['otdelka'] = flat.apply(lambda x: 1 if x.otdelka not in ['Не производится', None] else 0, axis=1)

train = pd.concat([train, pd.get_dummies(train['spalen'],prefix='sp')], axis=1)
test = pd.concat([test, pd.get_dummies(test['spalen'],prefix='sp')], axis=1)

# salestart = pd.to_datetime(pd.concat([test,train],ignore_index=False).reset_index(drop=True).groupby('id_bulk_sp')['date1'].min())
salestart = pd.to_datetime(flat.reset_index(drop=True).groupby('id_bulk_sp')['flat_startsale'].min())
train['dayfromstart'] = train.apply(lambda x: float((x['date1'] - salestart[x['id_bulk_sp']]).days), axis=1)
test['dayfromstart'] = test.apply(lambda x: float((x['date1'] - salestart[x['id_bulk_sp']]).days), axis=1)

flat.date_settle = flat.date_settle.fillna(flat.date_settle.min())
train['daytosettle'] = train.apply(lambda x: float(( flat[flat.id_bulk_sp == x['id_bulk_sp']]['date_settle'].iloc[0] - x['date1']).days), axis=1)
test['daytosettle'] = test.apply(lambda x: float(( flat[flat.id_bulk_sp == x['id_bulk_sp']]['date_settle'].iloc[0] - x['date1']).days), axis=1)

test['flat_count'] = test.apply(lambda x: len(flat[flat.id_bulk_sp == x['id_bulk_sp']]), axis=1)
train['flat_count'] = train.apply(lambda x: len(flat[flat.id_bulk_sp == x['id_bulk_sp']]), axis=1)

test['flat_left'] = test.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale >= x['date1'])]), axis=1) 
train['flat_left'] = train.apply(lambda x: len(flat[(flat.id_bulk_sp == x['id_bulk_sp'])&(flat.sale >= x['date1'])]), axis=1)

def getasaled(x,df,period):
    try:
        t = df[(df.id_bulk_sp == x['id_bulk_sp'])&(df.date1 == x['date1'] - DateOffset(months=period))].sort_values('date1').iloc[0]['value']
        return(t)
    except:
        return(None)
    
def getlastprice(x,df,period):
    try:
        t = df[(df.id_bulk_sp == x['id_bulk_sp'])&(df.date1 == x['date1'] - DateOffset(months=period))].sort_values('date1').iloc[0]['price']
        return(t)
    except:
        return(None)

##лаги продаж
train['saledlastmonth'] = train.apply(lambda x: getasaled(x,train,1), axis=1)
train['saledlast2month'] = train.apply(lambda x: getasaled(x,train,2), axis=1)
train['saledlast3month'] = train.apply(lambda x: getasaled(x,train,3), axis=1)
train['saledlast4month'] = train.apply(lambda x: getasaled(x,train,4), axis=1)
train['saledlast5month'] = train.apply(lambda x: getasaled(x,train,5), axis=1)
train['saledlast6month'] = train.apply(lambda x: getasaled(x,train,6), axis=1)

test['saledlastmonth'] = test.apply(lambda x: getasaled(x,train,1), axis=1)
test['saledlast2month'] = test.apply(lambda x: getasaled(x,train,2), axis=1)
test['saledlast3month'] = test.apply(lambda x: getasaled(x,train,3), axis=1)
test['saledlast4month'] = test.apply(lambda x: getasaled(x,train,4), axis=1)
test['saledlast5month'] = test.apply(lambda x: getasaled(x,train,5), axis=1)
test['saledlast6month'] = test.apply(lambda x: getasaled(x,train,6), axis=1)

##лаги цен
train['pricelastmonth'] = train.apply(lambda x: getlastprice(x,train,1), axis=1)
train['pricelast2month'] = train.apply(lambda x: getlastprice(x,train,2), axis=1)
train['pricelast3month'] = train.apply(lambda x: getlastprice(x,train,3), axis=1)
train['pricelast4month'] = train.apply(lambda x: getlastprice(x,train,4), axis=1)
train['pricelast5month'] = train.apply(lambda x: getlastprice(x,train,5), axis=1)
train['pricelast6month'] = train.apply(lambda x: getlastprice(x,train,6), axis=1)

test['pricelastmonth'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),1), axis=1)
test['pricelast2month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),2), axis=1)
test['pricelast3month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),3), axis=1)
test['pricelast4month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),4), axis=1)
test['pricelast5month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),5), axis=1)
test['pricelast6month'] = test.apply(lambda x: getlastprice(x,pd.concat([train,test]),6), axis=1)

##лаги остатков

train['prevcount'] = train.apply(lambda x: len(train[(train.date1 < x['date1'])&(train.id_bulk_sp == x['id_bulk_sp'])]), axis=1)
test['prevcount'] = test.apply(lambda x: len(pd.concat([train,test])[(pd.concat([train,test]).date1 < x['date1'])\
                                                                     &(pd.concat([train,test]).id_bulk_sp == x['id_bulk_sp'])]), axis=1)

train['otdelka'] = train.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['otdelka'].mean(), axis=1)
train['balcon'] = train.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['balcon'].mean(), axis=1)
test['otdelka'] = test.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['otdelka'].mean(), axis=1)
test['balcon'] = test.apply(lambda x: flat[flat.id_bulk_sp == x['id_bulk_sp']]['balcon'].mean(), axis=1)

euro = pd.read_csv('euro.csv',encoding='cp1251', sep=';')
euro['Средний курс'] = euro['Средний курс'].str.replace(',','.').astype(float)
euro['Месяц'] = pd.to_datetime(euro['Месяц'], errors='raise', dayfirst=True)
train['euro'] = train.apply(lambda x: euro[euro['Месяц'] == x['date1']]['Средний курс'].values[0], axis=1)
test['euro'] = test.apply(lambda x: euro[euro['Месяц'] == x['date1']]['Средний курс'].values[0], axis=1)

trainfull = train
testfull = test

train = train.drop(['id_bulk_sp', 'date1', 'id_bulk', 'value'], axis=1)
test = test.drop(['id_bulk_sp', 'date1', 'id_bulk'], axis=1)

cv = KFold(n_splits=5, shuffle=True, random_state=12)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.6/

CPU times: user 28min 32s, sys: 583 ms, total: 28min 32s
Wall time: 28min 32s


In [135]:
flat = pd.concat([flat,pd.get_dummies(flat.plan_size,prefix='plan')],axis=1)

In [143]:
test.to_csv('testfull.csv', index=False)
train.to_csv('trainfull.csv', index=False)

In [ ]:
add = pd.read_csv('train.csv',  usecols=['bulk_id', 'spalen', 'value'] ,encoding='cp1251')
add_test = pd.read_csv('test.csv',  usecols=['bulk_id', 'spalen'] ,encoding='cp1251')

In [ ]:
train1 = train.sort_index(axis=1)
test1 = test.sort_index(axis=1)

In [ ]:
def postproccesing(output):
    output.columns = ['id', 'value']
    output.value = output.value.apply(lambda x: 0 if x<0 else x)
    output['value'] = output.apply(lambda x: ((test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              if x['value']>(test.loc[x['id'],'mean_sq']*test.loc[x['id'],'flat_left']) 
                                              else x['value']), axis=1)
    df = pd.concat([testfull, output.value], axis=1)
    df['max'] = df.mean_sq*df.flat_left
    merged = pd.concat([df.groupby('id_bulk_sp')['value'].sum(),df.groupby('id_bulk_sp')['max'].mean()], axis=1)

    koef = merged[merged.value>merged['max']]['max']/merged[merged.value>merged['max']]['value']
    df['value'] = df.apply(lambda x: x['value'] if x['id_bulk_sp'] not in koef else x['value']*koef[x['id_bulk_sp']], axis=1)

    output = pd.DataFrame(data = df.value).reset_index()
    output.columns = ['id', 'value']
#     output.to_csv('result.csv', index=False)
    return(output)

In [ ]:
dollar_month = train1.groupby(train_cat.date1).dollar.mean()
dollar_month_test = test1.groupby(test_cat.date1).dollar.mean()
dollar_month = pd.concat([dollar_month,
dollar_month_test])

dollar_month = dollar_month.reset_index()
dollar_month = dollar_month.sort_values('date1')

dollar_month.loc[:, 'dollar_-1'] = dollar_month.dollar.shift(1)
dollar_month.loc[:, 'dollar_-2'] = dollar_month.dollar.shift(2)
dollar_month.loc[:, 'dollar_-3'] = dollar_month.dollar.shift(3)
dollar_month.loc[:, 'dollar_-4'] = dollar_month.dollar.shift(4)
dollar_month.loc[:, 'dollar_-5'] = dollar_month.dollar.shift(5)

dollar_month = dollar_month.drop('dollar', axis=1)

In [ ]:
train1 = pd.concat([
train1,
train_cat.loc[:, ['date1']].merge(dollar_month, how='left', on='date1')
    ], axis=1)

test1 = pd.concat([
test1,
test_cat.loc[:, ['date1']].merge(dollar_month, how='left', on='date1')
    ], axis=1)

## XGB

In [ ]:
print('xgb')
clf = XGBRegressor(n_estimators=2000, learning_rate=0.01, gamma=3, subsample=0.8, 
                   colsample_bytree=0.9, max_depth=6, nthread=-1)
clf.fit(train1.drop(['date1'], axis=1),y)
pr = clf.predict(test1.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output1_res = postproccesing(output)

In [ ]:
output1_res.to_csv('result_xgb_dollar_lags.csv', index=False)

## Catboost

In [ ]:
train2 = pd.concat([add.loc[:, ['bulk_id']], train1],axis=1)
test2 = pd.concat([add_test.loc[:, ['bulk_id']], test1],axis=1)

In [ ]:
np.where(np.in1d(train2.drop(['date1'], axis=1).columns, ['bulk_id', 'spalen']))

In [ ]:
print('cat')
cat = CatBoostRegressor(n_estimators=5000, l2_leaf_reg=8, learning_rate=0.015)
cat.fit(train2.drop(['date1'], axis=1),y, [0,63])
pr = cat.predict(test2.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output2_res = postproccesing(output)

In [ ]:
output2_res.to_csv('result_cat_categorical_5000.csv', index=False)

## Lightgbm

In [ ]:
print('lgb')
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=60,
                        max_depth = 6,
                        learning_rate=0.02,
                        n_estimators=2000,
                        gamma=3,
                       nthread=-1)

gbm.fit(train1.drop(['date1'], axis=1),y.values.reshape(-1))
pr = gbm.predict(test1.drop(['date1'], axis=1))
output = pd.DataFrame(data = pr).reset_index()
output3_res = postproccesing(output)

In [ ]:
output3_res.to_csv('result_lgb_3000.csv', index=False)

## Merge

In [ ]:
comparsion = pd.concat([output1_res.value, output2_res.value, output3_res.value], axis=1)
comparsion.columns = ['xgb', 'cat', 'lgb']
comparsion['mean1'] = ((7*comparsion.xgb + 7*comparsion.cat + 0*comparsion.lgb)/(7+7+0))
comparsion['mean2'] = ((7*comparsion.xgb + 7*comparsion.cat + 1*comparsion.lgb)/(7+7+1))

output = pd.DataFrame(data = comparsion['mean1']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_merge_categ_7_7_0_dollar_lag.csv', index=False)

output = pd.DataFrame(data = comparsion['mean2']).reset_index()
output.columns = ['id', 'value']
output.to_csv('result_merge_categ_7_7_1_dollar_lag.csv', index=False)